In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/product-position/upper.csv
/kaggle/input/product-position/upper_test.csv


In [2]:
df = pd.read_csv("../input/product-position/upper.csv", header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,1165,1166,1167,1168,1169,1170,1171,1172,1173,1174
0,0,-0.214777,-0.074414,0.079668,-0.041953,-0.715294,0.043155,0.754480,0.628223,-0.074455,...,0.088765,0.285463,-0.060605,-0.542769,-0.006594,0.665262,-0.015711,-0.667007,-0.013797,0.368727
1,0,-0.253997,0.000337,0.060563,-0.101104,-0.701834,0.070198,0.748214,0.656058,0.016878,...,0.182359,-0.046342,-0.233167,-0.234710,0.183400,0.591095,-0.059619,-0.819383,-0.283455,0.414261
2,0,-0.212620,-0.136841,0.113312,0.014819,-0.714882,-0.012742,0.720049,0.666651,-0.012994,...,0.020312,0.042942,-0.171825,-0.373853,-0.104966,0.448661,0.347881,-0.296482,-0.369788,-0.097517
3,0,-0.156624,-0.255746,0.086175,0.086967,-0.657194,0.011942,0.792931,0.718432,-0.084970,...,-0.142067,0.143729,-0.000996,-0.482060,-0.256660,0.559800,0.520810,-0.381071,-0.506321,-0.038203
4,0,-0.212018,0.003571,-0.026051,-0.032036,-0.683312,-0.024696,0.778626,0.821851,0.083669,...,-0.057536,0.141625,-0.370610,-0.451844,0.315462,0.651035,-0.119629,-0.642190,-0.087899,0.252129


In [3]:
df.shape

(2136, 1175)

In [4]:
df.groupby(0)[0].count()

0
0     343
1      87
2      29
3      13
4      52
5     105
6       2
7       2
8     741
9     433
10     91
11     62
12    163
13      4
14      9
Name: 0, dtype: int64

In [5]:
df_sel = df[(df[0]==0) | (df[0]==8) | (df[0]==9)]
df_sel.shape

(1517, 1175)

In [6]:
Xo = df_sel.drop([0],axis=1).values
yo = df_sel[0].values

In [7]:
Xo.shape

(1517, 1174)

In [8]:
yo.shape

(1517,)

In [9]:
np.unique(yo)

array([0, 8, 9])

In [10]:
yo[yo==8] = 1
yo[yo==9] = 2
np.unique(yo)

array([0, 1, 2])

In [11]:
[sum(yo == item) for item in np.unique(yo)]

[343, 741, 433]

In [12]:
from imblearn.under_sampling import RandomUnderSampler
sm = RandomUnderSampler(random_state=1)
X_res, y_res = sm.fit_resample(Xo,yo)

In [13]:
from sklearn.model_selection import train_test_split #มีข้อมูล 2 set
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.2, random_state=1)

In [14]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(X_train)
X_train_norm =  scaler.transform(X_train)
X_test_norm = scaler.transform(X_test)

In [15]:
import time
import sys
from time import sleep
from tqdm.notebook import tqdm
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import ExtraTreesClassifier

t0 = time.time()
pr = {
    'n_estimators' : [10,50,100,200,300],
    'min_samples_leaf' : [1,2,4,8,16,32],
}

for i in tqdm(pr):
    best_clf = GridSearchCV(ExtraTreesClassifier(random_state=0),pr,cv=10) 
    best_clf.fit(X_train_norm, y_train)
print ("Training time: " + str(round(time.time()-t0,2)) + " s")

  0%|          | 0/2 [00:00<?, ?it/s]

Training time: 198.32 s


In [16]:
yp = best_clf.predict(X_test_norm)
acc_test = sum(yp == y_test)/len(y_test)
print("Best score is " + str(best_clf.best_score_*100) +('\n') +"Best param is " + str(best_clf.best_params_))
print("Test Training Acc. : " + str(acc_test*100))

Best score is 98.66147516896856
Best param is {'min_samples_leaf': 1, 'n_estimators': 100}
Test Training Acc. : 98.54368932038835


In [17]:
df = pd.read_csv("../input/product-position/upper_test.csv", header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,1165,1166,1167,1168,1169,1170,1171,1172,1173,1174
0,0,-0.288798,0.123559,-0.145930,-0.215309,-0.185400,0.392191,0.387001,0.109418,-0.053896,...,-0.083072,-0.180611,-0.054263,-0.090253,-0.092374,0.204258,0.172720,-0.127758,-0.073426,-0.062349
1,0,-0.310982,0.278796,0.020278,-0.580068,-0.194813,0.513178,0.054633,-0.091677,0.446404,...,-0.249165,0.612952,0.460862,-0.594491,-0.101955,0.473182,-0.270727,-0.241867,0.267820,-0.319167
2,0,-0.232586,-0.001626,0.012946,-0.436545,-0.077329,0.669403,0.233532,-0.176750,0.085733,...,0.189612,0.061643,0.020043,0.020403,-0.160364,0.088831,0.255118,-0.239033,-0.312695,0.103348
3,0,-0.410512,0.074891,-0.350108,0.175275,0.275028,0.188904,0.001418,-0.026880,-0.095250,...,0.220428,0.527499,0.107231,-0.578609,-0.307208,0.289125,0.225836,-0.025417,-0.041300,-0.205327
4,0,-0.281948,0.228064,0.072771,-0.434250,-0.370780,0.324552,0.145079,0.034085,0.321535,...,-0.082343,0.070518,-0.496565,-0.673805,0.208622,0.616389,0.076365,-0.175746,-0.037616,-0.208103


In [18]:
df_sel = df[(df[0]==0) | (df[0]==8) | (df[0]==9)]
df_sel.shape

(634, 1175)

In [19]:
Xb = df_sel.drop([0],axis=1).values
yb = df_sel[0].values

In [20]:
yb[yb==8] = 1
yb[yb==9] = 2
np.unique(yb)

array([0, 1, 2])

In [21]:
X_blind_norm = scaler.transform(Xb)

In [22]:
yp = best_clf.predict(X_blind_norm)
acc_test = sum(yp == yb)/len(yb)
print("Best score is " + str(best_clf.best_score_*100))
print("Best param is " + str(best_clf.best_params_))
print("Test Training Acc. : " + str(acc_test*100))

Best score is 98.66147516896856
Best param is {'min_samples_leaf': 1, 'n_estimators': 100}
Test Training Acc. : 94.6372239747634
